In [1]:
!pip install pyspark[sql]
!pip install pyspark[pandas_on_spark] plotly
!pip install pyspark[connect]

In [2]:
#mengimport modul yang dibutuhkan
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import hash, abs

In [3]:
#membuat session
appName = "Sistem Penskoran Otomatis pada Soal Essay"
spark = SparkSession \
    .builder \
    .appName(appName) \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

# Import Dataset

In [4]:
df = spark.read.csv('/content/training_data_essay.csv', inferSchema=True, header=True, sep=';')
df.show()

+----------+------------+--------------------+----+-------------+
|       npm|nama_peserta|             jawaban|soal|skor_per_soal|
+----------+------------+--------------------+----+-------------+
|         0|       Admin|Tidak, Hanya memb...|   1|          100|
|         0|       Admin|Biaya dihitung be...|   2|          100|
|         0|       Admin|Hak cipta adalah ...|   3|          100|
|         0|       Admin|Dijelaskan kepada...|   4|          100|
|         0|       Admin|1. Melindungi dan...|   5|          100|
|         0|       Admin|Ruang Komputer, P...|   6|          100|
|         0|       Admin|Aturlah posisi pe...|   7|          100|
|         0|       Admin|Posisi Kepala dan...|   8|          100|
|         0|       Admin|1. Kecocokan soft...|   9|          100|
|         0|       Admin|1. Fokus dan expo...|  10|          100|
|         0|       Admin|1. Peralatan yang...|  11|          100|
|         0|       Admin|1. Dibuat grafik ...|  12|          100|
|112102003

# Pre-Processing Data

In [5]:
# Ganti koma (“,”) dengan titik (“.”) pada kolom “skor_per_soal”.
df1 = df.withColumn("skor_per_soal", regexp_replace(col("skor_per_soal"), ",", "."))
df1.show()

+----------+------------+--------------------+----+-------------+
|       npm|nama_peserta|             jawaban|soal|skor_per_soal|
+----------+------------+--------------------+----+-------------+
|         0|       Admin|Tidak, Hanya memb...|   1|          100|
|         0|       Admin|Biaya dihitung be...|   2|          100|
|         0|       Admin|Hak cipta adalah ...|   3|          100|
|         0|       Admin|Dijelaskan kepada...|   4|          100|
|         0|       Admin|1. Melindungi dan...|   5|          100|
|         0|       Admin|Ruang Komputer, P...|   6|          100|
|         0|       Admin|Aturlah posisi pe...|   7|          100|
|         0|       Admin|Posisi Kepala dan...|   8|          100|
|         0|       Admin|1. Kecocokan soft...|   9|          100|
|         0|       Admin|1. Fokus dan expo...|  10|          100|
|         0|       Admin|1. Peralatan yang...|  11|          100|
|         0|       Admin|1. Dibuat grafik ...|  12|          100|
|112102003

In [6]:
# Menampilkan skema DataFrame
df1.printSchema()

root
 |-- npm: integer (nullable = true)
 |-- nama_peserta: string (nullable = true)
 |-- jawaban: string (nullable = true)
 |-- soal: integer (nullable = true)
 |-- skor_per_soal: string (nullable = true)



In [7]:
# Ubah tipe data kolom "skor_per_soal" menjadi integer
data_essay = df1.withColumn("skor_per_soal", col("skor_per_soal").cast("float"))

data_essay.printSchema()

root
 |-- npm: integer (nullable = true)
 |-- nama_peserta: string (nullable = true)
 |-- jawaban: string (nullable = true)
 |-- soal: integer (nullable = true)
 |-- skor_per_soal: float (nullable = true)



# Menyiapkan Data

In [8]:
data = data_essay.select("soal","jawaban",'skor_per_soal')
data.show()

+----+--------------------+-------------+
|soal|             jawaban|skor_per_soal|
+----+--------------------+-------------+
|   1|Tidak, Hanya memb...|        100.0|
|   2|Biaya dihitung be...|        100.0|
|   3|Hak cipta adalah ...|        100.0|
|   4|Dijelaskan kepada...|        100.0|
|   5|1. Melindungi dan...|        100.0|
|   6|Ruang Komputer, P...|        100.0|
|   7|Aturlah posisi pe...|        100.0|
|   8|Posisi Kepala dan...|        100.0|
|   9|1. Kecocokan soft...|        100.0|
|  10|1. Fokus dan expo...|        100.0|
|  11|1. Peralatan yang...|        100.0|
|  12|1. Dibuat grafik ...|        100.0|
|   1|tidak, cuma mengi...|         52.7|
|   2|biaya dihitung be...|        42.86|
|   3|hak membuat merup...|        42.16|
|   4|dipaparkan pada k...|        27.19|
|   5|1. mencegah serta...|        44.14|
|   6|ruang komputer, p...|        100.0|
|   7|aturlah posisi fi...|        57.68|
|   8|posisi kepala ser...|        45.71|
+----+--------------------+-------

# Mengonversi kolom 'jawaban' menjadi representasi numerik

In [9]:
hash_data = data.withColumn("hashed_answer", hash("jawaban"))

hash_data.select("soal", "jawaban","skor_per_soal", "hashed_answer")
hash_data.show()

+----+--------------------+-------------+-------------+
|soal|             jawaban|skor_per_soal|hashed_answer|
+----+--------------------+-------------+-------------+
|   1|Tidak, Hanya memb...|        100.0|  -2059296905|
|   2|Biaya dihitung be...|        100.0|   1183180174|
|   3|Hak cipta adalah ...|        100.0|   1232762403|
|   4|Dijelaskan kepada...|        100.0|  -2035408785|
|   5|1. Melindungi dan...|        100.0|   1588395990|
|   6|Ruang Komputer, P...|        100.0|    339970513|
|   7|Aturlah posisi pe...|        100.0|     50850002|
|   8|Posisi Kepala dan...|        100.0|   -945877996|
|   9|1. Kecocokan soft...|        100.0|   1576366224|
|  10|1. Fokus dan expo...|        100.0|  -1905649442|
|  11|1. Peralatan yang...|        100.0|    550139146|
|  12|1. Dibuat grafik ...|        100.0|   1727767227|
|   1|tidak, cuma mengi...|         52.7|   1947733435|
|   2|biaya dihitung be...|        42.86|  -1139863335|
|   3|hak membuat merup...|        42.16|    122

# Memisahkan Data Training dan Data Testing

In [45]:
#membagi data, 70% training dan 30% testing
splits = hash_data.randomSplit([0.7, 0.3])
train = splits[0].withColumnRenamed("skor_per_soal", "label")
test = splits[1].withColumnRenamed("skor_per_soal", "trueLabel")

#menghitung baris data training dan testing
print("Jumlah baris data training :", train.count())
print("jumlah baris data testing :", test.count())

Jumlah baris data training : 86
jumlah baris data testing : 34


# Mendefinisikan Model ALS dan Mentrainingnya

In [46]:
#mendefinisikan algoritma ALS untuk sistem Penskoran Otomatis pada Soal Essay
als = ALS(maxIter=19, regParam=0.01, userCol="hashed_answer",
          itemCol="soal", ratingCol="label")

#mentraining model dengan fungsi ".fit()"
model = als.fit(train)
print("Model telah selesai ditraining!")

Model telah selesai ditraining!


# Melakukan prediksi menggunakan model yang telah ditraining terhadap data testing

In [47]:
prediction = model.transform(test)
prediction.show()

+----+--------------------+---------+-------------+----------+
|soal|             jawaban|trueLabel|hashed_answer|prediction|
+----+--------------------+---------+-------------+----------+
|   1|tidak, hanya memb...|    100.0|   -256638840|  99.99993|
|   1|tidak, hanya memb...|    100.0|   -256638840|  99.99993|
|   6|posisi tubuh, pos...|    90.37|  -1702735646|       NaN|
|   6|        posisi tubuh|    67.08|  -1240688658|       NaN|
|   6|ruang komputer, p...|    100.0|   1770907636|  99.99987|
|   6|ruang komputer, p...|    100.0|   1770907636|  99.99987|
|   6|ruang komputer, p...|    100.0|   1770907636|  99.99987|
|   3|hak cipta adalah ...|    83.43|  -1876419705|  83.42993|
|   3|emperbanyak cipta...|    47.67|    944450734|       NaN|
|   5|melindungi dan me...|    74.73|  -1932865057|  74.72994|
|   5|melindungi dan me...|    69.63|  -1210907486|       NaN|
|   5|memlihara sumber ...|     88.3|     40672765|       NaN|
|   4|Dijelaskan kepada...|    100.0|  -2035408785|    

# Evaluasi Model

In [48]:
evaluator = RegressionEvaluator(
    labelCol="trueLabel", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(prediction)
print ("Root Mean Square Error (RMSE):", rmse)

Root Mean Square Error (RMSE): nan


In [49]:
prediction.count()
a = prediction.count()
print("jumlah baris sebelum di hapus data kosong: ", a)
cleanPred = prediction.dropna(how="any", subset=["prediction"])
b = cleanPred.count()
print("jumlah baris setelah di hapus data kosong: ", b)
print("jumlah baris data kosong: ", a-b)

jumlah baris sebelum di hapus data kosong:  34
jumlah baris setelah di hapus data kosong:  13
jumlah baris data kosong:  21


In [50]:
cleanPred.show()

+----+--------------------+---------+-------------+----------+
|soal|             jawaban|trueLabel|hashed_answer|prediction|
+----+--------------------+---------+-------------+----------+
|   1|tidak, hanya memb...|    100.0|   -256638840|  99.99993|
|   1|tidak, hanya memb...|    100.0|   -256638840|  99.99993|
|   6|ruang komputer, p...|    100.0|   1770907636|  99.99987|
|   6|ruang komputer, p...|    100.0|   1770907636|  99.99987|
|   6|ruang komputer, p...|    100.0|   1770907636|  99.99987|
|   3|hak cipta adalah ...|    83.43|  -1876419705|  83.42993|
|   5|melindungi dan me...|    74.73|  -1932865057|  74.72994|
|   9|kecocokan softwar...|    65.89|    447829639| 65.889946|
|   4|dijelaskan kepada...|    72.06|  -1834227989| 72.059906|
|   8|posisi kepala dan...|    100.0|   1782344444|  99.99991|
|   7|aturlah posisi pe...|    86.22|  -1392782412|  86.21994|
|   2|biaya dihitung be...|    100.0|   1176853507|  99.99985|
|   2|biaya dihitung be...|    100.0|   1176853507|  99

In [51]:
rmse = evaluator.evaluate(cleanPred)
print("Root Mean Square Error (RMSE):", rmse)

Root Mean Square Error (RMSE): 0.00010312173339142772
